<a href="https://colab.research.google.com/github/rohanprakash7/BostonHousePricing_End_to_End/blob/main/RohanPrakash_LangChain_Templates_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 26px 28px 18px 28px; border-radius: 14px; margin-bottom: 22px; font-family: Arial, sans-serif;">
  <h2 style="margin-top: 0; font-size: 27px; letter-spacing: 0.5px;">
    🧠 LangChain Lab 2: Prompt Templates &amp; Memory
  </h2>
  <p style="font-size: 17px; margin-bottom: 8px;">
    <span style="color: #a5d8ff;">Instructor:</span> Prof. Dehghani
  </p>
  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 12px;">Lab Overview</h3>
  <p style="font-size: 16px; margin-bottom: 14px;">
    In this lab, you'll enhance your interactions with LLMs by using <b>Prompt Templates</b> and <b>Memory</b> features in LangChain.<br>
    You'll learn to create <b>structured prompts dynamically</b> and maintain <b>conversation history</b> across multiple turns.
  </p>
  <hr style="border: 1px solid #3f77d4; margin: 16px 0;">
  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 10px;">🎯 What You'll Learn</h3>
  <ul style="font-size: 16px; margin: 0 0 10px 22px; line-height: 1.8;">
    <li>🔹 <b>Prompt Templates</b> – Format inputs dynamically for LLMs.</li>
    <li>🔹 <b>Memory in LangChain</b> – Maintain context in multi-turn conversations.</li>
    <li>🔹 <b>Hands-on exercises</b> – Reinforce concepts with practical coding tasks.</li>
  </ul>
  <p style="font-size: 15.5px; margin-top: 8px;">
    By the end, you'll be able to structure prompts effectively and implement conversational memory in LangChain applications. 🚀
  </p>
</div>


##⚙️ Install essential packages

In [2]:
#⚙️ Install essential packages for LangChain with OpenAI & Gemini support

!pip install -q --upgrade langchain                # Core LangChain framework for building LLM workflows
!pip install -q --upgrade langchain-community      # Community integrations (still useful for many non-OpenAI/Gemini models)
!pip install -q --upgrade langchain-openai         # ✅ NEW: Dedicated package for OpenAI integrations
!pip install -q --upgrade langchain-google-genai   # Integration for Google's Gemini models
!pip install -q --upgrade openai                   # OpenAI SDK for native API calls (not strictly needed for LangChain, but often useful)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have 

##🔑 Step 2: Set Up OpenAI API Key

In [10]:
# ⚙️ Load API Keys from Colab Secrets
# ==================================

import os                                  # Used to set environment variables for API keys
from google.colab import userdata          # To securely access stored secrets in Colab

# Retrieve your stored secrets (API keys)
OPENAI_API_KEY = userdata.get('OPEN_AI_API')   # OpenAI API key for GPT models
GEMINI_API = userdata.get('GEMINI_API')   # Google Gemini API key for Gemini models

# Set environment variables for the APIs and confirm success
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY   # Set OpenAI key as environment variable
    print("✅ OpenAI API key loaded successfully!")
else:
    print("❌ OpenAI API key not found. Please set 'OPEN_AI_API' in Colab secrets.")

if GEMINI_API:
    os.environ["GEMINI_API"] = GEMINI_API   # Set Gemini key as environment variable
    print("✅ Google Gemini API key loaded successfully!")
else:
    print("❌ Google Gemini API key not found. Please set 'GEMINI_API_KEY' in Colab secrets.")

✅ OpenAI API key loaded successfully!
✅ Google Gemini API key loaded successfully!


<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 26px 28px 18px 28px; border-radius: 14px; margin-bottom: 22px; font-family: Arial, sans-serif;">
  <h2 style="margin-top: 0; font-size: 25px; letter-spacing: 0.5px;">📝 Prompt Templates in LangChain</h2>

  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 8px;">🔹 What are Prompt Templates?</h3>
  <p style="font-size: 16px; margin-bottom: 12px;">
    Prompt Templates let you <b>dynamically format prompts</b> by inserting variables, making interactions with LLMs more flexible and reusable.<br>
    Instead of writing static text, you can use placeholders that are filled in with real values at runtime.
  </p>

  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 8px;">🔹 Why Use Prompt Templates?</h3>
  <ul style="font-size: 16px; margin: 0 0 14px 22px; line-height: 1.8;">
    <li>✅ <b>Reusability</b> – No need for repetitive prompts.</li>
    <li>✅ <b>Dynamic Inputs</b> – Easily personalize prompts with new user data.</li>
    <li>✅ <b>Consistency</b> – Keeps your prompt formatting structured and reliable.</li>
  </ul>

  <h3 style="font-size: 17px; margin-bottom: 8px;">📌 Example Usage</h3>
  <div style="background: rgba(255,255,255,0.08); border-radius: 7px; padding: 11px 16px; margin-bottom: 6px;">
    <span style="color: #a5d8ff;">Static prompt:</span><br>
    <span style="font-family: 'Fira Mono', monospace; font-size: 15px; color: #fff;">
      "What are the benefits of AI in healthcare?"
    </span>
  </div>
  <div style="background: rgba(255,255,255,0.08); border-radius: 7px; padding: 11px 16px;">
    <span style="color: #a5d8ff;">Dynamic prompt with a template:</span><br>
    <span style="font-family: 'Fira Mono', monospace; font-size: 15px; color: #fff;">
      "What are the benefits of {technology} in {industry}?"
    </span><br>
    <span style="font-size: 15px;">If <b>{technology} = "AI"</b> and <b>{industry} = "healthcare"</b>, the prompt becomes:</span><br>
    <span style="font-family: 'Fira Mono', monospace; font-size: 15px; color: #fff;">
      "What are the benefits of AI in healthcare?"
    </span>
  </div>

  <p style="font-size: 16px; margin-top: 14px;">🚀 Let's get started with the first example!</p>
</div>


In [11]:
"""
# ==================================================
# 🎯 Using Prompt Templates with OpenAI (GPT-4)
# ==================================================
"""

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI   # ✅ NEW import for ChatOpenAI

# Step 1: Define a prompt template with variables
prompt_template = PromptTemplate(
    input_variables=["technology", "industry"],
    template="What are the benefits of {technology} in {industry} in 1 sentence?"
)

# Step 2: Format the prompt with specific values
formatted_prompt = prompt_template.format(technology="Drones", industry="SupplyChain")

# Step 3: Initialize the OpenAI LLM (GPT-4)
llm_ChatGPT = ChatOpenAI(model_name="gpt-4")

# Step 4: Generate the response
response_ChatGPT = llm_ChatGPT.invoke(formatted_prompt)

# Step 5: Display results
print("🔹 Generated Prompt:", formatted_prompt)
print("🔹 LLM Response:", response_ChatGPT.content)

🔹 Generated Prompt: What are the benefits of Drones in SupplyChain in 1 sentence?
🔹 LLM Response: Drones speed up the delivery process in supply chains, reduce human labor, lower transportation costs, and are capable of reaching inaccessible areas.


In [13]:
# ==================================================
# ✋ **Hands-On: Creating Dynamic Prompt Templates**
# ==================================================

# 📌 **Task Instructions:**
# 1️⃣ Fill in the missing placeholders (-----) to complete the code.
# 2️⃣ Ensure the Prompt Template correctly replaces {topic} and {context}.
# 3️⃣ Run the code and verify GPT-4 generates a response.

# ✅ Step 1: Define a Prompt Template
prompt_template = PromptTemplate(
    input_variables=["topic", "context"],  # Fill in the missing variable name
    template="How does {topic} impact {context} in a few words?"  # Structure of the prompt
)

# ✅ Step 2: Format the prompt with actual values
formatted_prompt = prompt_template.format(topic="weight training", context="gym")

# ✅ Step 3: Generate a response using OpenAI (GPT-4)
llm_ChatGPT = ChatOpenAI(model_name="gpt-4")  # Initialize the ChatGPT model
response_ChatGPT = llm_ChatGPT.invoke(formatted_prompt)  # Fill in the correct variable for invoke

# ✅ Step 4: Display results
print("🔹 **Generated Prompt:**", formatted_prompt)
print("🔹 **LLM Response:**", response_ChatGPT.content)  # Extract response content


🔹 **Generated Prompt:** How does weight training impact gym in a few words?
🔹 **LLM Response:** Weight training enhances strength, improves body composition, promotes fat loss, boosts metabolism, increases athletic performance, and supports bone health in the gym.


In [14]:
"""
# ==================================================
# 🔄 Using Prompt Templates in a Loop
# ==================================================
"""

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI  # ✅ NEW import—no deprecation warning!

# Step 1️⃣: Define a prompt template with variables
prompt_template = PromptTemplate(
    input_variables=["technology", "industry"],
    template="In one sentence, how does {technology} impact {industry} in 1 sentence?"
)

# Step 2️⃣: Initialize the OpenAI LLM (GPT-4)
llm_ChatGPT = ChatOpenAI(model_name="gpt-4")

# Step 3️⃣: Define input values for the loop
input_data = [
    {"technology": "AI", "industry": "education"},
    {"technology": "Blockchain", "industry": "finance"},
    {"technology": "5G", "industry": "telecommunications"},
]

# Step 4️⃣: Loop through inputs, format the prompt, and generate a response
for data in input_data:
    formatted_prompt = prompt_template.format(**data)
    response_ChatGPT = llm_ChatGPT.invoke(formatted_prompt)

    # Step 5️⃣: Display results in a clear, modern format
    print(f"🔹 Prompt: {formatted_prompt}")
    print(f"💡 Response: {response_ChatGPT.content}")
    print("-" * 60)

🔹 Prompt: In one sentence, how does AI impact education in 1 sentence?
💡 Response: AI impacts education by personalizing learning experiences, automating administrative tasks, and providing educators with tools to improve student understanding and engagement.
------------------------------------------------------------
🔹 Prompt: In one sentence, how does Blockchain impact finance in 1 sentence?
💡 Response: Blockchain technology transforms finance by providing a decentralized, secure and transparent system for transactions, reducing costs, increasing efficiency and eliminating the need for intermediaries.
------------------------------------------------------------
🔹 Prompt: In one sentence, how does 5G impact telecommunications in 1 sentence?
💡 Response: 5G revolutionizes telecommunications by offering significantly faster data speeds, lower latency, increased device capacity and improved reliability.
------------------------------------------------------------


<div style="background: linear-gradient(135deg, #1a386e 0%, #377ce8 100%); color: white; padding: 24px 26px 16px 26px; border-radius: 12px; margin-bottom: 22px; font-family: Arial, sans-serif;">
  <h3 style="margin-top: 0; font-size: 21px;">🔗 Wrapping Up: Why Prompt Templates Matter</h3>
  <p style="font-size: 16px;">
    Just as you wouldn't rewrite a whole menu for every customer in a coffee shop, you don't need to create a new prompt for every question you ask an LLM.
    <br><br>
    <b>Prompt templates</b> give you a reusable, flexible, and structured way to interact with language models—making your code cleaner, your queries more consistent, and your applications easier to scale.
    <br><br>
    Whether you're building a chatbot, automating business tasks, or analyzing data, prompt templates are an essential tool in your GenAI toolkit!
  </p>
</div>


<div style="background: linear-gradient(135deg, #001a70 0%, #0055d4 100%); color: white; padding: 26px 28px 18px 28px; border-radius: 14px; margin-bottom: 22px; font-family: Arial, sans-serif;">
  <h2 style="margin-top: 0; font-size: 27px; letter-spacing: 0.5px;">
    🧠 Understanding Memory in LangChain
  </h2>

  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 10px;">🔹 What is Memory in LangChain?</h3>
  <p style="font-size: 16px; margin-bottom: 13px;">
    By default, LLMs <b>do not remember past interactions</b>.<br>
    LangChain <b>Memory</b> allows an AI model to <b>retain context</b> across multiple turns, enabling more natural, conversational interactions.
  </p>

  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 10px;">🔹 Why Use Memory?</h3>
  <ul style="font-size: 16px; margin: 0 0 14px 22px; line-height: 1.8;">
    <li>✅ <b>Maintains conversation history</b> – AI can recall previous exchanges.</li>
    <li>✅ <b>Improves response coherence</b> – Reduces redundant user re-explanations.</li>
    <li>✅ <b>Essential for chatbots &amp; agents</b> – Allows multi-turn dialogue without loss of context.</li>
  </ul>

  <h3 style="color: #a5d8ff; font-size: 19px; margin-bottom: 10px;">🔹 Types of Memory in LangChain</h3>
  <ul style="font-size: 16px; margin: 0 0 14px 22px; line-height: 1.7;">
    <li>1️⃣ <b>ConversationBufferMemory</b> – Stores messages in a buffer (basic memory).</li>
    <li>2️⃣ <b>ConversationSummaryMemory</b> – Summarizes past interactions instead of storing all messages.</li>
    <li>3️⃣ <b>ConversationBufferWindowMemory</b> – Retains only the last N interactions for efficiency.</li>
    <li>4️⃣ <b>Vector-based Memory</b> – Uses embeddings for advanced retrieval of past conversations.</li>
  </ul>

  <h3 style="font-size: 17px; margin-bottom: 8px;">🚀 What We'll Do in This Lab</h3>
  <p style="font-size: 16px;">
    We’ll start with <b>ConversationBufferMemory</b>, which allows an LLM to <b>recall past messages</b> and interact in a more natural, memory-enhanced way.<br>
    <br>
    <span style="color: #a5d8ff;">Note:</span> When using a conversation chain with memory, you’ll use the <b><code>predict()</code></b> method instead of <code>invoke()</code>. This lets the AI maintain and use context across multiple turns.<br>
    <br>
    Let's get started! 👇
  </p>
</div>


In [15]:
"""
# ==================================================
# 💬🧠 Memory Matters: AnniversaryBot Demo (Stateless vs. Memory)
# ==================================================
#
# This demo shows how LangChain's memory feature allows an AI assistant to remember
# details—using the example of a user telling the bot their anniversary date.
"""

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# -------------------------------
# 1️⃣ Version WITHOUT Memory (Stateless)
# -------------------------------
llm_stateless = ChatOpenAI(model_name="gpt-4")

print("\n========== WITHOUT MEMORY (Stateless LLM) ==========")
print("👩‍❤️‍👨 User: Our anniversary is October 5th. Please remember that!")
response1 = llm_stateless.invoke("Our anniversary is October 5th. Please remember that!")
print("🤖 AnniversaryBot:", response1.content)

print("👩‍❤️‍👨 User: When is our anniversary?")
response2 = llm_stateless.invoke("When is our anniversary?")
print("🤖 AnniversaryBot:", response2.content)

# -------------------------------
# 2️⃣ Version WITH Memory
# -------------------------------
memory = ConversationBufferMemory()
llm_with_mem = ChatOpenAI(model_name="gpt-4")
conversation_with_mem = ConversationChain(llm=llm_with_mem, memory=memory)

print("\n========== WITH MEMORY ==========")
print("👩‍❤️‍👨 User: Our anniversary is October 5th. Please remember that!")
response3 = conversation_with_mem.predict(input="Our anniversary is October 5th. Please remember that!")
print("🤖 AnniversaryBot:", response3)

print("👩‍❤️‍👨 User: When is our anniversary?")
response4 = conversation_with_mem.predict(input="When is our anniversary?")
print("🤖 AnniversaryBot:", response4)



========== WITHOUT MEMORY (Stateless LLM) ==========
👩‍❤️‍👨 User: Our anniversary is October 5th. Please remember that!
🤖 AnniversaryBot: I will certainly remember that! Happy early anniversary! If you need any help planning or organizing anything for the special day, feel free to ask.
👩‍❤️‍👨 User: When is our anniversary?
🤖 AnniversaryBot: As an AI, I don't have personal relationships or anniversaries. But I was launched by OpenAI in June 2020.

========== WITH MEMORY ==========
👩‍❤️‍👨 User: Our anniversary is October 5th. Please remember that!


/tmp/ipython-input-2823701360.py:31: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipython-input-2823701360.py:33: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation_with_mem = ConversationChain(llm=llm_with_mem, memory=memory)


🤖 AnniversaryBot: Of course, I will keep a note of it! October 5th is your anniversary. Should I remind you a few days before the date or on the day itself?
👩‍❤️‍👨 User: When is our anniversary?
🤖 AnniversaryBot: Your anniversary is on October 5th.


In [16]:
# ==================================================
# ✋ Hands-On: Using Memory with OpenAI (Beer Game - Supply Chain Predictions)
# ==================================================
#
# 📌 Task Instructions:
# 1️⃣ Fill in the missing placeholders (-----) to complete the code.
# 2️⃣ Ensure the AI remembers previous demand data and predicts future order quantities.
# 3️⃣ Run the code and check if ChatGPT maintains context for supply chain decisions.

# ✅ Step 1: Initialize Memory
memory = ConversationBufferMemory()  # Initialize the correct memory class

# ✅ Step 2: Initialize ChatGPT with Memory
llm_ChatGPT = ChatOpenAI(model_name="gpt-4")  # Initialize ChatGPT model
conversation = ConversationChain(llm=llm_ChatGPT, memory=memory)  # Attach memory to conversation

# ✅ Step 3: Run Multiple Interactions
print("\n💬 **Retailer:** Last week, the customer demand was 200 units. What should I order this week?")
response_ChatGPT = conversation_with_mem.predict(input="Last week, the customer demand was 200 units. What should I order this week?")  # Call the correct method
print("🤖 **ChatGPT:**", response_ChatGPT)

print("\n💬 **Retailer:** If demand increases by 10%, how many units should I prepare for next week?")
response_ChatGPT = conversation_with_mem.predict(input="If demand increases by 10%, how many units should I prepare for next week?")
print("🤖 **ChatGPT:**", response_ChatGPT)

print("\n💬 **Retailer:** What was the demand I mentioned last week?")
response_ChatGPT = conversation_with_mem.predict(input="What was the demand I mentioned last week?")
print("🤖 **ChatGPT:**", response_ChatGPT)



💬 **Retailer:** Last week, the customer demand was 200 units. What should I order this week?
🤖 **ChatGPT:** As an AI, I don't have real-time data to predict the exact customer demand for this week. However, generally, businesses use sales forecasting methods relying on historical data and current market trends. If last week's demand was 200 units and you have no reason to anticipate a significant change, a similar amount could be a safe starting point. It's always a good idea to consult with your sales team or use predictive analytics tools for more accurate forecasting.

💬 **Retailer:** If demand increases by 10%, how many units should I prepare for next week?
🤖 **ChatGPT:** If demand increases by 10% and last week's demand was 200 units, you should prepare for an increase of 20 units (10% of 200). So, you would need a total of 220 units for next week.

💬 **Retailer:** What was the demand I mentioned last week?
🤖 **ChatGPT:** Last week, you mentioned that the customer demand was 200 

## Using 'Summarized Conversation' Example


In [17]:
# ==================================================
# 🎤 **Using Memory in LangChain: Job Interview Prep**
# ==================================================
#
# This script simulates a job interview practice session.
# It uses ConversationSummaryMemory to retain key points from previous exchanges
# rather than storing the full conversation history.

# ✅ Import required classes
from langchain.memory import ConversationSummaryMemory  # Summarized conversation memory

# ✅ Step 1: Initialize Memory
# This memory will maintain a **summarized** version of the conversation.
memory = ConversationSummaryMemory(llm=ChatOpenAI(model_name="gpt-4"))

# ✅ Step 2: Initialize ChatGPT with Memory
llm = ChatOpenAI(model_name="gpt-4")  # Using GPT-4 model

# ✅ Step 3: Initialize Conversation Chain
# The model will summarize key details from the job interview practice.
conversation = ConversationChain(llm=llm, memory=memory)

# ✅ Step 4: Conduct the Interview Simulation

print("\n💬 **User:** Can you ask me a common interview question?")
response = conversation.predict(input="Can you ask me a common interview question?")
print("🤖 **ChatGPT:**", response)

# ✅ Check memory after first interaction
print("\n📜 **Memory Summary After 1st Question:**")
print(memory.load_memory_variables({})["history"])

print("\n💬 **User:** My biggest strength is adaptability and problem-solving.")
response = conversation.predict(input="My biggest strength is adaptability and problem-solving.")
print("🤖 **ChatGPT:**", response)

# ✅ Check memory after user shares strength
print("\n📜 **Memory Summary After Strength Response:**")
print(memory.load_memory_variables({})["history"])

print("\n💬 **User:** My biggest weakness is that I sometimes overthink decisions.")
response = conversation.predict(input="My biggest weakness is that I sometimes overthink decisions.")
print("🤖 **ChatGPT:**", response)

# ✅ Check memory after user shares weakness
print("\n📜 **Memory Summary After Weakness Response:**")
print(memory.load_memory_variables({})["history"])

print("\n💬 **User:** Can you summarize what we discussed so far?")
response = conversation.predict(input="Can you summarize what we discussed so far?")
print("🤖 **ChatGPT:**", response)

# ✅ Final Memory Check
print("\n📜 **Final Memory Summary:**")
print(memory.load_memory_variables({})["history"])


/tmp/ipython-input-809088591.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=ChatOpenAI(model_name="gpt-4"))



💬 **User:** Can you ask me a common interview question?
🤖 **ChatGPT:** Sure, one of the common interview questions is: Can you describe a challenging situation you encountered at work and how you handled it?

📜 **Memory Summary After 1st Question:**
The human asks the AI to pose a common interview question. The AI asks the human to describe a challenging situation they encountered at work and how they handled it.

💬 **User:** My biggest strength is adaptability and problem-solving.
🤖 **ChatGPT:** That's a great strength to have, especially in today's ever-changing work environment. Could you provide a specific example where you had to adapt quickly and solve a problem at your workplace?

📜 **Memory Summary After Strength Response:**
The human asks the AI to pose a common interview question. The AI inquires about the human's response to a challenging workplace situation. The human responds by stating their biggest strength is adaptability and problem-solving, to which the AI asks for a

In [18]:
# ==================================================
# 🍺 **LangChain Beer Game: Comparing Memory Types
# ==================================================
#
# This script simulates a Beer Game ordering process over 6 weeks.
# It uses:
# 1️⃣ ConversationBufferMemory (Tracks full history)
# 2️⃣ ConversationBufferWindowMemory (Tracks only last 3 orders)
#
# The AI predicts the next order quantity based on past interactions.

# ✅ Import required libraries
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate

# ✅ Step 1: Initialize Memory Types
buffer_memory = ConversationBufferMemory(return_messages=True)  # Stores entire history
window_memory = ConversationBufferWindowMemory(k=3, return_messages=True)  # Stores last 3 interactions

# ✅ Step 2: Initialize Chat Model (NEW!)
llm = ChatOpenAI(model="gpt-4")

# ✅ Step 3: Define a Prompt Template
beer_game_template = PromptTemplate(
    input_variables=["context"],
    template="""
    You are managing a supply chain for a beer distribution system.
    Orders fluctuate at first but stabilize later.

    {context}

    Based on past orders, what should be the next order quantity?
    """
)


# ✅ Step 4: Define Processing Pipelines
buffer_chain = beer_game_template | llm
window_chain = beer_game_template | llm

# ✅ Step 5: Define Order Fluctuations (First 3 weeks volatile, last 3 weeks stable)
weekly_orders = [20, 50, 10, 25, 30, 30]  # Example fluctuations

# Store results for comparison
buffer_memory_log = []
window_memory_log = []
buffer_predictions = []
window_predictions = []

# ✅ Step 6: Run the Simulation
for week in range(1, len(weekly_orders) + 1):
    prev_orders = ", ".join(map(str, weekly_orders[:week]))  # Orders seen so far
    context = f"Week {week}: The previous orders were {prev_orders}."

    # Store input in memory
    buffer_memory.save_context({"context": context}, {"response": ""})
    window_memory.save_context({"context": context}, {"response": ""})

    # Get AI predictions using RunnableSequence
    buffer_prediction = buffer_chain.invoke({"context": context})
    window_prediction = window_chain.invoke({"context": context})

    # Retrieve memory states
    buffer_memory_summary = buffer_memory.load_memory_variables({})["history"]
    window_memory_summary = window_memory.load_memory_variables({})["history"]

    # Store memory states and predictions
    buffer_memory_log.append(buffer_memory_summary)
    window_memory_log.append(window_memory_summary)
    buffer_predictions.append(buffer_prediction.content)
    window_predictions.append(window_prediction.content)

# ✅ Step 7: Display Results in a Table
df = pd.DataFrame({
    "Week": list(range(1, len(weekly_orders) + 1)),
    "Actual Orders": weekly_orders,
    "Buffer Memory (Stores All)": buffer_memory_log,
    "Window Memory (Last 3 Turns)": window_memory_log,
    "Buffer Memory Prediction": buffer_predictions,
    "Window Memory Prediction": window_predictions
})


/tmp/ipython-input-1262663304.py:20: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=3, return_messages=True)  # Stores last 3 interactions


In [19]:
display(df)

,Week,Actual Orders,Buffer Memory (Stores All),Window Memory (Last 3 Turns),Buffer Memory Prediction,Window Memory Prediction
0,1,20,[content='Week 1: The previous orders were 20....,[content='Week 1: The previous orders were 20....,Without more information provided such as week...,"As a language model AI developed by OpenAI, I ..."
1,2,50,[content='Week 1: The previous orders were 20....,[content='Week 1: The previous orders were 20....,The order quantity seems to be increasing. Whi...,The order quantity trend is increasing. Howeve...
2,3,10,[content='Week 1: The previous orders were 20....,[content='Week 1: The previous orders were 20....,The next order quantity would ideally be based...,The average order quantity over the past three...
3,4,25,[content='Week 1: The previous orders were 20....,"[content='Week 2: The previous orders were 20,...",The next order quantity is difficult to predic...,The order quantities seem relatively unpredict...
4,5,30,[content='Week 1: The previous orders were 20....,"[content='Week 3: The previous orders were 20,...",Although I am an AI trained for text predictio...,This depends entirely on the method used for f...
5,6,30,[content='Week 1: The previous orders were 20....,"[content='Week 4: The previous orders were 20,...",One common strategy used in forecasting future...,Making predictions based on past orders can of...


In [20]:
# ✅ Save the table to an Excel file
df.to_excel("beer_game_memory_comparison.xlsx", index=False)

# ✅ Print confirmation message
print("Data saved to 'beer_game_memory_comparison.xlsx'")


Data saved to 'beer_game_memory_comparison.xlsx'


# 📌 **Assignment: AI Stock Market Trend Prediction with Memory**

## **Objective**
In this assignment, you will use AI to predict stock market trends based on historical stock prices. You will compare how different memory types affect AI's ability to track and predict future trends.

## **Tasks**
1. **Initialize memory types** (`ConversationBufferMemory` and `ConversationBufferWindowMemory`).
2. **Define the AI model** (GPT-4 or another suitable model).
3. **Complete the prompt template** to guide AI predictions.
4. **Process stock price data** and use memory to store past trends.
5. **Retrieve and analyze stored memory** after each step.
6. **Invoke the AI model correctly** to generate predictions.
7. **Save results to an Excel file** for analysis.

## **Expected Outcome**
You will observe how AI predictions change when it has full history vs. limited memory. This will help you understand the impact of memory in AI-based forecasting.

🚀 **Complete the placeholders and run the script to generate insights!** 🚀


In [24]:
# ==================================================
# ✋ **Hands-On: Creating Dynamic Prompt Templates**
# 📈 **AI Assignment: Stock Market Trend Prediction with Memory**
# ==================================================
#
# 🔹 In this assignment, you will use AI to predict stock market trends.
# 🔹 You will compare how different memory types affect AI's ability to track stock price movements.
# 🔹 Complete the placeholders (----) to make the script functional.
#
# 📌 **Your Tasks:**
# 1️⃣ Initialize the correct memory types.
# 2️⃣ Define the AI model.
# 3️⃣ Complete the template prompt.
# 4️⃣ Use memory correctly when processing stock data.
# 5️⃣ Ensure correct invocation of AI for predictions.
# 6️⃣ Retrieve and analyze stored memory.
# 7️⃣ Save results in an Excel file.

# ✅ Import required libraries
import pandas as pd
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory  # Import appropriate memory classes
from langchain_openai import ChatOpenAI  # Import ChatGPT model
from langchain.prompts import PromptTemplate  # Import PromptTemplate
# ✅ Step 1: Initialize Memory Types
buffer_memory = buffer_memory = ConversationBufferMemory(memory_key="history", return_messages=True)  # Stores full stock history
window_memory = window_memory = ConversationBufferWindowMemory(k=3, memory_key="history", return_messages=True)  # Stores last 3 stock movements

# ✅ Step 2: Initialize Chat Model
llm = ChatOpenAI(model="gpt-4") # Define the AI model (GPT-4 or another model)

# ✅ Step 3: Define a Prompt Template
stock_prediction_template = PromptTemplate(
    input_variables=["context"],
    template="""
    You are an AI financial analyst predicting stock market trends.

    {context}

    Based on this stock price history, what will be the next trend (Up, Down, or Stable)?
    """
)

# ✅ Step 4: Define Processing Pipelines
buffer_chain = buffer_chain = stock_prediction_template | llm # Define how memory connects to AI
window_chain = buffer_chain = stock_prediction_template | llm # Define how memory connects to AI with windowed memory

# ✅ Step 5: Define Stock Price Data (Fluctuations in the first weeks, then stabilizing)
stock_prices = [120, 125, 110, 130, 128, 129]  # Example price movements

# Store results for comparison
buffer_memory_log = []
window_memory_log = []
buffer_predictions = []
window_predictions = []

# ✅ Step 6: Run the Prediction Simulation
for week in range(1, len(stock_prices) + 1):
    prev_prices = ", ".join(map(str, stock_prices[:week]))  # Stocks seen so far
    context = f"Week {week}: The previous stock prices were {prev_prices}."

    # Store input in memory
    buffer_memory.chat_memory.add_user_message(context)  # Store context in buffer memory
    window_memory.chat_memory.add_user_message(context)  # Store context in windowed memory

    # Get AI predictions
    buffer_prediction = buffer_chain.invoke({"context": context}) # Invoke AI for buffer memory
    window_prediction = window_chain.invoke({"context": context})  # Invoke AI for window memory

    # Retrieve memory states
    buffer_memory_summary = buffer_memory.load_memory_variables({})["history"]
    window_memory_summary = window_memory.load_memory_variables({})["history"]

    # Store memory states and predictions
    buffer_memory_log.append(buffer_memory_summary)
    window_memory_log.append(window_memory_summary)
    buffer_predictions.append(buffer_prediction.content)
    window_predictions.append(window_prediction.content)

# ✅ Step 7: Save Results in an Excel File
df = pd.DataFrame({
    "Week": list(range(1, len(stock_prices) + 1)),
    "Stock Price": stock_prices,
    "Buffer Memory (Stores All)": buffer_memory_log,
    "Window Memory (Last 3 Turns)": window_memory_log,
    "Buffer Memory Prediction": buffer_predictions,
    "Window Memory Prediction": window_predictions
})

df.to_excel("stock_market_memory_comparison.xlsx", index=False)


print("Assignment completed! Data saved to 'stock_market_memory_comparison.xlsx'")


Assignment completed! Data saved to 'stock_market_memory_comparison.xlsx'
